In [34]:
!pip install rasa==1.10.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
!pip install -U tables==3.6.1
!pip install -U ipython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
 !python -m spacy download en

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 5.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [37]:
!pip install nest_asyncio==1.3.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
import os
import rasa
import nest_asyncio
nest_asyncio.apply()
print("event loop ready")

event loop ready


In [39]:
from rasa.cli.scaffold import create_initial_project


In [40]:
project = 'test project'
create_initial_project(project)

In [41]:
os.chdir(project)
print(os.listdir("."))

['data', 'tests', 'credentials.yml', '__init__.py', 'domain.yml', '__pycache__', 'actions.py', 'config.yml', 'endpoints.yml']


In [42]:
config = "config.yml"
training_files = "data/"
domain = "domain.yml"
output = "models/"

print(config,training_files,domain,output)

config.yml data/ domain.yml models/


In [43]:
model_path = rasa.train(domain,config,[training_files],output)
print(model_path)

Processed Story Blocks: 100%|██████████| 5/5 [00:00<00:00, 127.94it/s, # trackers=24]

Training Core model...



Processed trackers: 100%|██████████| 5/5 [00:00<00:00, 109.36it/s, # actions=16]
Processed actions: 16it [00:00, 582.98it/s, # examples=16]
Epochs: 100%|██████████| 100/100 [00:15<00:00,  6.36it/s, t_loss=0.071, loss=0.003, acc=1.000]


Core model training completed.
Training NLU model...


/usr/local/lib/python3.7/dist-packages/rasa/utils/common.py:363: UserWarning: You specified 'DIET' to train entities, but no entities are present in the training data. Skip training of entities.
Epochs: 100%|██████████| 100/100 [00:17<00:00,  5.84it/s, t_loss=1.592, i_loss=0.205, i_acc=1.000]


NLU model training completed.
Your Rasa model is trained and saved at '/content/test project/test project/models/20220614-084434.tar.gz'.
models/20220614-084434.tar.gz


In [44]:
from rasa.jupyter import chat

endpoints = "endpoints.yml"

chat(model_path,endpoints)

Your bot is ready to talk! Type your messages here or send '/stop'.
/stop


In [45]:
%%writefile data/nlu.nd

## intent:book_room
-i want to book a room
-i need a room
-i am looking for a room
-is room empty
-book a room for a night.

## intent:num_room
-I need [2](number) rooms
-is [3](number) room available
-[4](number) rooms
-can i get the [3](number) rooms
-I want [3](number) rooms. 

## intent:simple
- i want a simple room
- simple
- can i get the simple room
- i would like a simple room

## intent:deluxe
- i want a deluxe room
- deluxe
- can i get the deluxe room
- i would like a deluxe room

## intent:greet
- hey
- hello
- hi
- good morning
- good evening
- hey there

## intent:goodbye
- bye
- goodbye
- see you around
- see you later

## intent:affirm
- yes
- indeed
- of course
- that sounds good
- correct

## intent:deny
- no
- never
- I don't think so
- don't like that
- no way
- not really

## intent:mood_great
- perfect
- very good
- great
- amazing
- wonderful
- I am feeling very good
- I am great
- I'm good

## intent:mood_unhappy
- sad
- very sad
- unhappy
- bad
- very bad
- awful
- terrible
- not very good
- extremely sad
- so sad

## intent:bot_challenge
- are you a bot?
- are you a human?
- am I talking to a bot?
- am I talking to a human?



Writing data/nlu.nd


In [46]:
%%writefile domain.yml

intents:
 
  - book_room
  - num_room
  - simple
  - deluxe
  - greet
  - goodbye
  - affirm
  - deny
  - mood_great
  - mood_unhappy
  - bot_challenge

responses:

  utter_book_room:
  - text: "How many rooms would you like to book?"

  utter_num_room:
  - text: "What type of room do you want, Simple or Deluxe?"

  utter_simple:
  - text: "You have chosen a simple room"

  utter_deluxe:
  - text: "You have chosen a deluxe room"

  utter_greet:
  - text: "Hey! How are you?"

  utter_cheer_up:
  - text: "Here is something to cheer you up:"
    image: "https://i.imgur.com/nGF1K8f.jpg"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_happy:
  - text: "Great, carry on!"

  utter_goodbye:
  - text: "Bye"

  utter_iamabot:
  - text: "I am a bot, powered by Rasa."

entities:
  - number

session_config:
  session_expiration_time: 60
  carry_over_slots_to_new_session: true


Overwriting domain.yml


In [47]:
%%writefile data/stories.md

## booking simple room 
* greet 
  - utter_greet
* book_room
  - utter_book_room
* num_room
  - utter_num_room
* simple
  - utter_simple
  
## booking deluxe room 
* greet 
  - utter_greet
* book_room
  - utter_book_room
* num_room
  - utter_num_room
* deluxe
  - utter_deluxe

## happy path
* greet
  - utter_greet
* mood_great
  - utter_happy

## sad path 1
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* affirm
  - utter_happy

## sad path 2
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* deny
  - utter_goodbye

## say goodbye
* goodbye
  - utter_goodbye

## bot challenge
* bot_challenge
  - utter_iamabot


Overwriting data/stories.md


In [49]:
%%writefile credentials.YAML_VERSION

# This file contains the credentials for the voice & chat platforms
# which your bot is using.
# https://rasa.com/docs/rasa/user-guide/messaging-and-voice-channels/

#rest:
#  # you don't need to provide anything here - this channel doesn't
#  # require any credentials


telegram:
  verify: "Droiid_bot"
  webhook_url:  "https://4a75-2402-3a80-1342-572d-951-9f1b-bcd0-2dc3.in.ngrok.io"
  access-token: "5413935620:AAFmcG4TOySTXmSkbnuAzgkC0iVF_oHNhhU"

#slack:
#  slack_token: "<your slack token>"
#  slack_channel: "<the slack channel>"

#socketio:
#  user_message_evt: <event name for user message>
#  bot_message_evt: <event name for but messages>
#  session_persistence: <true/false>

#mattermost:
#  url: "https://<mattermost instance>/api/v4"
#  token: "<bot token>"
#  webhook_url: "<callback URL>"

# This entry is needed if you are using Rasa X. The entry represents credentials 
# for the Rasa X "channel", i.e. Talk to your bot and Share with guest testers.
#rasa:
  #url: "http://localhost:5002/api"



Overwriting credentials.YAML_VERSION


In [50]:
model_path = rasa.train(domain,config,[training_files],output)
print(model_path)

Processed Story Blocks: 100%|██████████| 7/7 [00:00<00:00, 108.32it/s, # trackers=26]

Training Core model...



Processed trackers: 100%|██████████| 7/7 [00:00<00:00, 75.84it/s, # actions=24]
Processed actions: 24it [00:00, 618.97it/s, # examples=24]
Epochs: 100%|██████████| 100/100 [00:23<00:00,  4.27it/s, t_loss=0.111, loss=0.030, acc=1.000]


Core model training completed.
NLU data/configuration did not change. No need to retrain NLU model.
Your Rasa model is trained and saved at '/content/test project/test project/models/20220614-084631.tar.gz'.
models/20220614-084631.tar.gz
